# Libraries

In [9]:
#Use the os package to interact with the environment
import os

#Use the sys package to interact with stderr 
import sys

#Use the FireCloud API to interact with the workspace from firecloud 
import api as fapi

#Use the BigQuery client library instead of pandas-gbq for greater control over query jobs from google.cloud 
import bigquery

#Import the iPython HTML rendering for displaying links to Google Cloud Console from IPython.core.display 
#import display, HTML

#Import urllib modules for building URLs to Google Cloud Console 
import urllib.parse

#Bring in Pandas for Dataframe functionality 
import pandas as pd

import time
from platform import python_version

print(python_version())
print('finsihed loading libraries')

3.7.9
finsihed loading libraries


# Utility functions

In [16]:
# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_bq(description, link_text, bq_dataset, bq_table=None):
    project, dataset = bq_dataset.split('.', 1)
    if bq_table:
        page_params = {'page': 'table', 'p': project, 'd': dataset, 't': bq_table}
    else:
        page_params = {'page': 'dataset', 'p': project, 'd': dataset}
    
    url = '{}?{}'.format(
        'https://console.cloud.google.com/bigquery',
        urllib.parse.urlencode(page_params))

    display_html_link(description, link_text, url)
    
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}')
    !$command

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))
    
# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Get the data from a query
def bq_query(query):
    """Return the contents of a query against BigQuery"""
    return pd.read_gbq(
        query,
        project_id=BILLING_PROJECT_ID,
        dialect='standard')

print('finished')

finished


# Global

In [18]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']


#WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})
if 1:
    GS_RELEASE_PATH = 'gs://amp-pd-data/releases/2019_v1release_1015'
    GS_CLINICAL_RELEASE_PATH = f'{GS_RELEASE_PATH}/clinical'

    GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2019_v1release_1015/wgs'
    GS_WGS_RELEASE_PLINK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'plink')
    GS_WGS_RELEASE_GATK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'gatk')

    BQ_RELEASE_DATASET = 'amp-pd-research.2019_v1release_1015'
    
    BQ_WGS_RELEASE_DATASET = BQ_RELEASE_DATASET
    BQ_WGS_RELEASE_DATASET_STD = BQ_WGS_RELEASE_DATASET.replace(':', '.')
    
    #output
    #BQ_TIER1_RELEASE_DATASET_STD = amp-pd-research.2019_v1release_1015
    #BQ_TIER2_RELEASE_DATASET_STD = amp-pd-research.2019_v1release_1015_tier2
    #BQ_WGS_RELEASE_DATASET_STD = amp-pd-research.2019_v1release_1015_genomics

    print(BILLING_PROJECT_ID)
    print(WORKSPACE_BUCKET)
    print(GS_CLINICAL_RELEASE_PATH)
    #print(GS_WGS_RELEASE_PLINK_PATH)
    #print(GS_WGS_RELEASE_GATK_PATH)

googleGSId='fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5'    
    
print('finsihed')

larobak11
gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5
gs://amp-pd-data/releases/2019_v1release_1015/clinical
finsihed


In [19]:
cmd='pwd'
shell_do(cmd)

Executing: pwd
/home/jupyter-user/notebooks/LSDProject/edit


In [11]:
#make a new directory for data using the command line
dataDir='/home/jupyter-user/notebooks/LSDProject/data/'
if not os.path.exists(dataDir):
    os.makedirs(dataDir)
#make a tools directory
toolsDir='/home/jupyter-user/notebooks/LSDProject/tools/'
if not os.path.exists(toolsDir):
    os.makedirs(toolsDir)
print('dataDir:', dataDir)
print('toolsDir:', toolsDir)

#install plink if not already
plinkCmd=toolsDir+'plink'
if os.path.isfile(plinkCmd) == False:
    print('plink cmd not found')
    cmd='wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip -P ' + toolsDir
    print('cmd:', cmd)
    shell_do(cmd)

dataDir: /home/jupyter-user/notebooks/LSDProject/data/
toolsDir: /home/jupyter-user/notebooks/LSDProject/tools/


# Copy AMP-PD plink files

In [8]:
cmd=f'gsutil -u {BILLING_PROJECT_ID} cp {GS_WGS_RELEASE_PLINK_PATH}/bfile/all_vcfs* {dataDir}'
print('cmd:', cmd)
shell_do(cmd)

cmd: gsutil -u larobak11 cp gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs* /home/jupyter-user/notebooks/LSDProject/data/
Executing: gsutil -u larobak11 cp gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs* /home/jupyter-user/notebooks/LSDProject/data/
Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bed...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bim...
Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.fam...
Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.log...
- [4 files][ 21.4 GiB/ 21.4 GiB]    5.7 MiB/s                                

In [6]:
#just checking here
#copy vcf files
#shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_WGS_RELEASE_PATH}')
#shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_WGS_RELEASE_GATK_PATH}')
#shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile')
#make a directory similar to github
#shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {GS_WGS_RELEASE_PLINK_PATH}/bfile/all_vcfs* ~/bin/data_temp')

#ORIG
#cmd=f'gsutil -u {BILLING_PROJECT_ID} cp {GS_WGS_RELEASE_PLINK_PATH}/bfile/all_vcfs* {dataDir}'
#cmd=f'gsutil -u {BILLING_PROJECT_ID} ls gs://amp-pd-data/releases/2019_v1release_1015/'
#print('cmd:', cmd)
#shell_do(cmd)

GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2019_v1release_1015/wgs'
#cmd=f'gsutil -u {BILLING_PROJECT_ID} cp {GS_WGS_RELEASE_PLINK_PATH}/bfile/all_vcfs* {dataDir}'
#cmd=f'gsutil -u {BILLING_PROJECT_ID} ls gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/'
cmd=f'gsutil -u {BILLING_PROJECT_ID} ls gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/'
print('cmd:', cmd)
shell_do(cmd)

#gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/gatk/vcf/

#gs://amp-pd-genomics/releases/
#amp-pd-research.2019_v1release_1015
#cmd=f'gsutil -u {BILLING_PROJECT_ID} cp gs://amp-pd-data/releases/2019_v1release_1015/wgs_gatk_joint_genotyping_samples.csv {dataDir}'
#print('cmd:', cmd)
#shell_do(cmd)

#copy to bucket
#shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp gs://amp-pd-data/releases/2019_v1release_1015/wgs_sample_inventory.csv {WORKSPACE_BUCKET}')
#print('finished copying')
#shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp {GS_WGS_RELEASE_PLINK_PATH}/bfile/all_vcfs* {dataDir}')
#print('Done !!')


#shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp gs://amp-pd-data/releases/2019_v1release_1015/clinical/UPSIT_dictionary.csv {dataDir}')
#print('finished')

cmd: gsutil -u larobak11 ls gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/
Executing: gsutil -u larobak11 ls gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/README.txt
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bed
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bim
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.fam
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.log


# Set covars

In [20]:
covariates = f"""

SELECT 
participant_id, sex, age_at_baseline, race

FROM `{BQ_RELEASE_DATASET}.Demographics`

"""

covs = bq_query(covariates)
#dim
print(type(covs))
print(covs.head())
#covs.shape

#pheno
phenotype = f"""

SELECT 
* 

FROM `{BQ_RELEASE_DATASET}.amp_pd_case_control`

"""

pheno = bq_query(phenotype)
pheno.head()


Downloading: 100%|██████████| 4298/4298 [00:01<00:00, 3405.86rows/s]


<class 'pandas.core.frame.DataFrame'>
  participant_id   sex  age_at_baseline   race
0       PP-41564  Male               22  White
1  PD-PDZV843ATF  Male               24  White
2  PD-PDCK871NBR  Male               24  White
3       PP-51718  Male               25  White
4       PP-56267  Male               25  White


Downloading: 100%|██████████| 4298/4298 [00:01<00:00, 3403.09rows/s]


,participant_id,diagnosis_at_baseline,diagnosis_latest,case_control_other_at_baseline,case_control_other_latest
0,PD-PDAB597LKL,Parkinsonism,Parkinsonism,Other,Other
1,PD-PDCT406CN9,Parkinsonism,Parkinsonism,Other,Other
2,PP-54461,Prodromal motor PD,Idiopathic PD,Other,Case
3,PP-54215,Prodromal motor PD,Idiopathic PD,Other,Case
4,PP-18567,Prodromal motor PD,Idiopathic PD,Other,Case


# Check clinical to get study ID

In [24]:
covariates = f"""

SELECT 
*

FROM `{BQ_RELEASE_DATASET}.amp_pd_participants`

"""

clinCov = bq_query(covariates)
print(clinCov.head())
print(clinCov.shape)

print('Done')

Downloading: 100%|██████████| 4298/4298 [00:01<00:00, 2867.45rows/s]

      participant_id            guid study study_participant_id
0  HB-PD_INVAA223GY7  PD_INVAA223GY7   HBS       PD_INVAA223GY7
1  HB-PD_INVAA769TBF  PD_INVAA769TBF   HBS       PD_INVAA769TBF
2  HB-PD_INVAB465GYE  PD_INVAB465GYE   HBS       PD_INVAB465GYE
3  HB-PD_INVAC618MF7  PD_INVAC618MF7   HBS       PD_INVAC618MF7
4  HB-PD_INVAD033HX2  PD_INVAD033HX2   HBS       PD_INVAD033HX2
(4298, 4)
Done


In [11]:
#set up covs in plink format
print('covs:\n', covs.head())
covsp=pd.DataFrame()
covsp['FID']=covs['participant_id']
covsp['IID']=covs['participant_id']
covsp['sex']=covs['sex']
covsp['AgeBaseLine']=covs['age_at_baseline']
covsp['race']=covs['race']
covsp.sex[(covsp.sex == "Male")] = 1
covsp.sex[(covsp.sex == "Female")] = 2
print('covsp:\n', covsp.head())
print(covsp.shape)
    
print('finished')

covs:
   participant_id   sex  age_at_baseline   race
0       PP-41564  Male               22  White
1  PD-PDZV843ATF  Male               24  White
2  PD-PDCK871NBR  Male               24  White
3       PP-51718  Male               25  White
4       PP-56267  Male               25  White
covsp:
              FID            IID sex  AgeBaseLine   race
0       PP-41564       PP-41564   1           22  White
1  PD-PDZV843ATF  PD-PDZV843ATF   1           24  White
2  PD-PDCK871NBR  PD-PDCK871NBR   1           24  White
3       PP-51718       PP-51718   1           25  White
4       PP-56267       PP-56267   1           25  White
(4298, 5)
finished


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
#set up pheno
phenop = pheno
phenop['IID'] = phenop['participant_id']
phenop = phenop[['participant_id', 'IID', 'case_control_other_latest']]
phenop.columns = ['FID','IID', 'PHENO']
phenop = phenop[(phenop.PHENO != 'Other')]
phenop.PHENO[(phenop.PHENO == "Case")] = 2
phenop.PHENO[(phenop.PHENO == "Control")] = 1

phenop.head()
print(phenop.shape)

(4005, 3)


In [15]:
#write
plinkCovsFile=dataDir+ 'plink_covs.tab'
plinkPhenoFile=dataDir+'plink_pheno.tab'
with open(plinkCovsFile, 'w') as f:
    f.write(covsp.to_csv(index=False, sep='\t'))
    
with open(plinkPhenoFile, 'w') as f:
    f.write(phenop.to_csv(index=False, sep='\t'))
    
print('finished')

finished


In [18]:
#Now we'll use Plink's --update-sex command to update the fam file with 
#the information we queried from the demographics file.
cmd=plinkCmd+' --bfile '+ dataDir + 'all_vcfs' + ' --update-sex ' + plinkCovsFile + ' --make-bed ' + ' --out ' + dataDir + 'plink_updated_sex' + ' --threads 2'
print('cmd:', cmd)
shell_do(cmd)
print('Done')


cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/all_vcfs --update-sex /home/jupyter-user/notebooks/LSDProject/data/plink_covs.tab --make-bed  --out /home/jupyter-user/notebooks/LSDProject/data/plink_updated_sex --threads 2
Executing: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/all_vcfs --update-sex /home/jupyter-user/notebooks/LSDProject/data/plink_covs.tab --make-bed  --out /home/jupyter-user/notebooks/LSDProject/data/plink_updated_sex --threads 2
PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter-user/notebooks/LSDProject/data/plink_updated_sex.log.
Options in effect:
  --bfile /home/jupyter-user/notebooks/LSDProject/data/all_vcfs
  --make-bed
  --out /home/jupyter-user/notebooks/LSDProject/data/plink_updated_sex
  --threads 2
  --up

# Find PCA

In [23]:
#plink --bfile prune --pca --out PPMI.PCA
shell_do(f'{plinkCmd} --bfile {dataDir}plink_updated_sex --pca --out {dataDir}plink_updated_sex.PCA')
#this prodcues the file: plink_updated_sex.PCA.eigenvec which is used to add as phenotype
print('Done !!')

Executing: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data//plink_updated_sex --pca --out /home/jupyter-user/notebooks/LSDProject/data//plink_updated_sex.PCA
PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter-user/notebooks/LSDProject/data//plink_updated_sex.PCA.log.
Options in effect:
  --bfile /home/jupyter-user/notebooks/LSDProject/data//plink_updated_sex
  --out /home/jupyter-user/notebooks/LSDProject/data//plink_updated_sex.PCA
  --pca

15043 MB RAM detected; reserving 7521 MB for main workspace.
28791969 variants loaded from .bim file.
3074 people (1721 males, 1353 females) loaded from .fam.
Using up to 4 threads (change this with --threads).
Before main variant filters, 3074 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



13729800 markers complete.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



14426880 markers complete.

...

15087360 markers complete.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



15763200 markers complete.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Relationship matrix calculation complete.
--pca: Results saved to and eigenvectors]
/home/jupyter-user/notebooks/LSDProject/data//plink_updated_sex.PCA.eigenval
and
/home/jupyter-user/notebooks/LSDProject/data//plink_updated_sex.PCA.eigenvec .
Done !!


# Add PCA to phenotype file

In [7]:
#read PCA file
inFile=dataDir+'/'+ 'plink_updated_sex.PCA.eigenvec'
print('inFile:', inFile)
pcaDf=pd.read_csv(inFile, sep=" ", header=None)
print('pcaDf\n',pcaDf.head())
print('shape:', pcaDf.shape)
count=1
pcNames=[]
while count <= 20:
    s='PC'+str(count)
    pcNames.append(s)
    count=count+1
#print('names:',pcNames)
#add column names
colNames=['FID', 'IID']
colNames.extend(pcNames)
#print('colNames:', colNames)
pcaDf.columns=colNames
del pcaDf['IID']#cleaner
#check header names 
print(pcaDf.columns)
#read the pheno type again
inFile=dataDir+'/plink_pheno.tab'
phenoDf=pd.read_csv(inFile, sep="\t")
print('phenoDf\n',phenoDf.head())
#read the covar file
inFile=dataDir+'/plink_covs.tab'
covDf=pd.read_csv(inFile, sep="\t")
print('covDf\n',covDf.head())
merDf=phenoDf.merge(covDf, left_on='FID', right_on='FID')
del merDf['IID_y']
merDf.rename({'IID_x': 'IID'}, axis=1, inplace=True)
print('\nmerDf\n', merDf.head())#4005

#merge with PCA
merDf_2=pcaDf.merge(merDf, left_on='FID', right_on='FID')
print('merDf 2 shape:',merDf_2.shape)
print(merDf_2.head())
#re check this for a better way of chaning column positions
v = merDf_2[['FID','IID','PHENO','sex','AgeBaseLine','race','PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9',
       'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18','PC19', 'PC20']]
print('\n v\n',v.head())  
#write
fileName=dataDir+'/plink_pheno_withPCs.tab'
with open(fileName, 'w') as f:
    f.write(v.to_csv(index=False, sep='\t'))

    
print('Done')

inFile: /home/jupyter-user/notebooks/LSDProject/data//plink_updated_sex.PCA.eigenvec
pcaDf
         0        1         2         3             4         5         6   \
0  BF-1001  BF-1001 -0.003598  0.000104 -1.192030e-05  0.000095 -0.000181   
1  BF-1002  BF-1002 -0.003540 -0.000012  1.159970e-05 -0.000038 -0.000036   
2  BF-1003  BF-1003 -0.003353  0.000060  9.764960e-07 -0.000137 -0.000123   
3  BF-1004  BF-1004 -0.003481 -0.000020 -1.434620e-05 -0.000146 -0.000168   
4  BF-1005  BF-1005 -0.003584  0.000068  1.296540e-05 -0.000012 -0.000047   

         7         8         9   ...        12        13        14        15  \
0 -0.000064  0.000154 -0.000092  ...  0.000231  0.000123  0.000025  0.000007   
1  0.000017  0.000031  0.000032  ...  0.000108  0.000060 -0.000147 -0.000193   
2 -0.000036 -0.000108 -0.000013  ... -0.000057  0.000098 -0.000084 -0.000075   
3  0.000030  0.000007 -0.000158  ...  0.000049  0.000055  0.000014  0.000064   
4  0.000111 -0.000064 -0.000007  ...  0.00005

# Set up for LSD genes

In [11]:
#mkdir /home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
phenoFile=dataDir+'/plink_pheno_withPCs.tab'
#hg38 coordinates of the genes
rangeFile=dataDir+'LSD-54-genes_plinkFormat-fromBed_Aug0720.txt'
#check i file exists and if it does not copy it from google cloud
if os.path.isfile(rangeFile) == False:
    print('range file not found')
    cmd='gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-54-genes_plinkFormat-fromBed_Aug0720.txt '+dataDir
    print('cmd:', cmd)
    shell_do(cmd)

print('Done !!')
           
#check the file in the working diorectory and see how many variants we have using wc -l LSDGenes_WGS_AMPPD.bim

range file not found
cmd: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-54-genes_plinkFormat-fromBed_Aug0720.txt /home/jupyter-user/notebooks/LSDProject/data/
Executing: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-54-genes_plinkFormat-fromBed_Aug0720.txt /home/jupyter-user/notebooks/LSDProject/data/
Copying gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-54-genes_plinkFormat-fromBed_Aug0720.txt...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 1 objects/1.4 KiB.                                      
Done !!


# Run plink to extract/subet the LSD genes variants

In [16]:
plinkPrefix='LSDGenes-WGS-AMPPD'
# Extract/subet the LSD genes variants; hg38 coordinates
cmd=plinkCmd+' --bfile ' + dataDir + 'plink_updated_sex' + ' --extract range ' + rangeFile + ' --make-bed --out ' +dataDir + plinkPrefix
print('cmd:', cmd)
shell_do(cmd)
#shell_do(f'~/bin/plink --bfile {dataDir}/plink_updated_sex --extract range {rangeFile} --make-bed --out {workingDir}/{plinkPrefix}')
#check the file in the working diorectory and see how many variants we have using wc -l LSDGenes_WGS_AMPPD.bim


print('Done !!')
           


cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/plink_updated_sex --extract range /home/jupyter-user/notebooks/LSDProject/data/LSD-54-genes_plinkFormat-fromBed_Aug0720.txt --make-bed --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD
Executing: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/plink_updated_sex --extract range /home/jupyter-user/notebooks/LSDProject/data/LSD-54-genes_plinkFormat-fromBed_Aug0720.txt --make-bed --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD
PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD.log.
Options in effect:
  --bfile /home/jupyter-user/notebooks/LSDProject/data/plink_updated_sex
  --extract range /home/jupyter-us

# Set up results directory

In [7]:
resDir='/home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/'
if os.path.isdir(resDir)==False:
    cmd='mkdir ' + resDir
    shell_do(cmd)
print('Done')

Done


# RUN --MODEL AND --ASSOC IN PLINK with no covaraiates

In [26]:
#model
#plinkPrefix='LSDGenes_WGS_AMPPD'
outPrefix='LSDGenes-WGS-AMPPD_noCov'
phenoFile=dataDir+'plink_pheno_withPCs.tab'
#shell_do(f'~/bin/plink --bfile {workingDir}/{plinkPrefix} --pheno {phenoFile} --pheno-name PHENO --model --out {workingDir}/{outPrefix}')
cmd=plinkCmd+' --bfile ' + dataDir + plinkPrefix + ' --pheno ' + phenoFile + ' --pheno-name PHENO --model --out ' + resDir+outPrefix

print('cmd:', cmd)
#shell_do(cmd)
print('Finished model')


#assoc
#shell_do(f'~/bin/plink --bfile {workingDir}/{plinkPrefix} --assoc --pheno {phenoFile} --pheno-name PHENO --ci 0.95 --out {workingDir}/{outPrefix}')
outPrefix='LSDGenes-WGS-AMPPD_Assoc-noCov'
cmd=plinkCmd+ ' --bfile ' + dataDir + plinkPrefix + '  --assoc --pheno ' + phenoFile + ' --pheno-name PHENO --ci 0.95 --out ' +  resDir+outPrefix               
print('cmd:',cmd)
shell_do(cmd)

print('Finished assoc')

cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --model --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes-WGS-AMPPD_noCov
Finished model
cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD  --assoc --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --ci 0.95 --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes-WGS-AMPPD_Assoc-noCov
Executing: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD  --assoc --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --ci 0.95 --out /home/jupyter-user/notebooks/LSDProject/result

# Generate Vcfs from plink files for LSD

In [35]:
plinkPrefix='LSDGenes-WGS-AMPPD'
vcfOutPrefix='LSDGenes-WGS-VCF'
cmd=plinkCmd + ' --bfile ' + dataDir + plinkPrefix + ' --pheno ' + phenoFile + ' --pheno-name PHENO --recode vcf-fid --out ' + dataDir + vcfOutPrefix
print('cmd:', cmd)
shell_do(cmd)
#shell_do(f'~/bin/plink --bfile {workingDir}/{plinkPrefix} --pheno {phenoFile} --pheno-name PHENO --recode vcf-fid --out {workingDir}/{plinkPrefix}')
print('Done')

cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --recode vcf-fid --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF
Executing: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --recode vcf-fid --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF
PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.log.
Options in effect:
  --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD
  --out /home/jupyter-user/notebooks/LSDProje

# Generate Vcfs from plink files for LSD (bgzip)

In [37]:
cmd='bgzip ' + dataDir + vcfOutPrefix + '.vcf'
print('cmd:', cmd)
shell_do(cmd)
#shell_do(f'bgzip {workingDir}/{plinkPrefix}.vcf')
print('Done bgzip')

cmd: bgzip /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf
Executing: bgzip /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf
Done bgzip


# Generate Vcfs from plink files for LSD (tabix)

In [39]:
cmd='tabix -f -p vcf ' + dataDir + vcfOutPrefix + '.vcf.gz'
print('cmd:', cmd)
shell_do(cmd)

#shell_do(f'tabix -f -p vcf {workingDir}/{plinkPrefix}.vcf.gz')
print('Done tabix')


cmd: tabix -f -p vcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf.gz
Executing: tabix -f -p vcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf.gz
Done tabix


# Copy Annovar command

In [45]:
cmd = 'gsutil cp gs://' + googleGSId+'/notebooks/annovar.latest.tar.gz ' + toolsDir + 'annovar'
print('cmd:', cmd)
shell_do(cmd)

#for some reason tar from here does not work, so do it on the command line
#cmd='tar –xvzf ' + toolsDir + 'annovar/annovar.latest.tar.gz'
#print('cmd:', cmd)
#shell_do(cmd)

cmd: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/annovar.latest.tar.gz /home/jupyter-user/notebooks/LSDProject/tools/annovar
Executing: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/annovar.latest.tar.gz /home/jupyter-user/notebooks/LSDProject/tools/annovar
Copying gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/annovar.latest.tar.gz...
| [1 files][135.1 MiB/135.1 MiB]                                                
Operation completed over 1 objects/135.1 MiB.                                    
cmd: tar –xvzf /home/jupyter-user/notebooks/LSDProject/tools/annovar/annovar.latest.tar.gz
Executing: tar –xvzf /home/jupyter-user/notebooks/LSDProject/tools/annovar/annovar.latest.tar.gz
tar: invalid option -- '�'
Try 'tar --help' or 'tar --usage' for more information.


# Set up annovar DB

In [ ]:
#under the annovar tools dir run the belwo commands to set up the databases for annotation
#./annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20200316 humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03  humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp35a humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp150 humandb/
#./annotate_variation.pl -buildver hg38 -downdb -webfrom annovar revel  humandb/


# Run Annovar

In [55]:
humanDB='/home/jupyter-user/notebooks/LSDProject/tools/annovar/annovar/humandb'
outPrefix='LSD-GENES.annovar'
annovarCmd='/home/jupyter-user/notebooks/LSDProject/tools/annovar/annovar/table_annovar.pl'
#workingDir=LSDGenes
plinkPrefix='LSDGenes-WGS-AMPPD'
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#vcfFile=workingDir+'/'+plinkPrefix+'.vcf.gz'
vcfFile=dataDir+vcfOutPrefix+'.vcf.gz'

cmd1= annovarCmd+' '+vcfFile+' '+humanDB+' -buildver hg38 ' + '-out ' + resDir + outPrefix
cmd2=' -remove -protocol refGene,cytoband,dbnsfp35a,avsnp150,revel,gnomad211_genome,clinvar_20200316 -operation g,r,f,f,f,f,f -nastring . -vcfinput -polish '
cmd=cmd1+cmd2
print('cmd:', cmd)
start_time = time.time()
shell_do(cmd)
end_time = time.time()
process_time = end_time - start_time
print('time:', process_time)
#shell_do(f'{annovarCmd} {vcfFile} \
#{humanDB} -buildver hg38 \
#-out {workingDir}/{outPrefix} \
#-remove -protocol refGene,cytoband,dbnsfp35a,avsnp150,revel,gnomad211_genome,clinvar_20200316 \
#-operation g,r,f,f,f,f,f -nastring . -vcfinput -polish')

print('finished annovar')

cmd: /home/jupyter-user/notebooks/LSDProject/tools/annovar/annovar/table_annovar.pl /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf.gz /home/jupyter-user/notebooks/LSDProject/tools/annovar/annovar/humandb -buildver hg38 -out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSD-GENES.annovar -remove -protocol refGene,cytoband,dbnsfp35a,avsnp150,revel,gnomad211_genome,clinvar_20200316 -operation g,r,f,f,f,f,f -nastring . -vcfinput -polish 
Executing: /home/jupyter-user/notebooks/LSDProject/tools/annovar/annovar/table_annovar.pl /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf.gz /home/jupyter-user/notebooks/LSDProject/tools/annovar/annovar/humandb -buildver hg38 -out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSD-GENES.annovar -remove -protocol refGene,cytoband,dbnsfp35a,avsnp150,revel,gnomad211_genome,clinvar_20200316 -operation g,r,f,f,f,f,f -nastring . -vcfinput -polish 

NOTICE: Running with system comma

In [60]:
#just checking
#inFile='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes/LSD-GENES.annovar.hg38_multianno.txt'
inFile=resDir+'LSD-GENES.annovar.hg38_multianno.txt'
print('inFile:', inFile)
tmpDf=pd.read_csv(inFile, sep="\t")
print(tmpDf.shape)
print(tmpDf.head())
print('Done')

inFile: /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSD-GENES.annovar.hg38_multianno.txt
(34355, 3191)
   Chr     Start       End Ref Alt Func.refGene Gene.refGene  \
0    1  16985990  16985990   C   T       exonic      ATP13A2   
1    1  16986063  16986063   A   G       exonic      ATP13A2   
2    1  16986091  16986091   G   A       exonic      ATP13A2   
3    1  16986097  16986097   G   A       exonic      ATP13A2   
4    1  16986101  16986101   T   A       exonic      ATP13A2   

  GeneDetail.refGene ExonicFunc.refGene  \
0                  .  nonsynonymous SNV   
1                  .     synonymous SNV   
2                  .  nonsynonymous SNV   
3                  .  nonsynonymous SNV   
4                  .  nonsynonymous SNV   

                                AAChange.refGene  ... Otherinfo3077  \
0  ATP13A2:NM_001141974:exon27:c.G3472A:p.V1158M  ...           0/0   
1  ATP13A2:NM_001141974:exon27:c.T3399C:p.V1133V  ...           0/0   
2  ATP13A2:NM_0

# Extract coding variants

In [9]:
annoPrefix='LSD-GENES'
annoFile=resDir+'/'+annoPrefix+'.annovar.hg38_multianno.txt'
shell_do(f'grep "exonic"  {annoFile} > {dataDir}CODING_regions.txt')
shell_do(f'cut -f 1,2,3,7  {dataDir}CODING_regions.txt > {dataDir}CODING.txt')
print('Done !!')

Executing: grep "exonic"  /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120//LSD-GENES.annovar.hg38_multianno.txt > /home/jupyter-user/notebooks/LSDProject/data/CODING_regions.txt
Executing: cut -f 1,2,3,7  /home/jupyter-user/notebooks/LSDProject/data/CODING_regions.txt > /home/jupyter-user/notebooks/LSDProject/data/CODING.txt
Done !!


# Prep the files and generate PLINK binary with only coding variants

In [11]:
bFile=dataDir+'LSDGenes-WGS-AMPPD'
outPrefix=dataDir+'LSDGenes-WGS-AMPPD_exonic'
phenoFile=dataDir+'plink_pheno_withPCs.tab'
cmd=plinkCmd+ ' --bfile ' + bFile + ' --pheno ' + phenoFile + ' --pheno-name PHENO --extract range ' + dataDir + 'CODING.txt' + ' --make-bed --out ' + outPrefix
print('cmd:', cmd)
shell_do(cmd)
#shell_do(f'~/bin/plink --bfile {bFile} --pheno {phenoFile} --pheno-name PHENO --extract range {workingDir}/CODING.txt --make-bed --out {outPrefix}')
print('Done !!')

cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --extract range /home/jupyter-user/notebooks/LSDProject/data/CODING.txt --make-bed --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_exonic
Executing: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --extract range /home/jupyter-user/notebooks/LSDProject/data/CODING.txt --make-bed --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_exonic
PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-W

# Create the VCF with only coding variants

In [12]:
bFile=dataDir+'/'+'LSDGenes-WGS-AMPPD_exonic'
outPrefix=dataDir+'/'+'LSDGenes-WGS-AMPPD_exonic'
shell_do(f'{plinkCmd} --bfile {bFile} --pheno {phenoFile} --pheno-name PHENO --recode vcf-fid --out {outPrefix}')
shell_do(f'bgzip {dataDir}/LSDGenes-WGS-AMPPD_exonic.vcf')
shell_do(f'tabix -f -p vcf {dataDir}/LSDGenes-WGS-AMPPD_exonic.vcf.gz')
print('Done !!')

Executing: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_exonic --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --recode vcf-fid --out /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_exonic
PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_exonic.log.
Options in effect:
  --bfile /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_exonic
  --out /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_exonic
  --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab
  --pheno-name PHENO
  --recode vcf-fid

15043 MB RAM detected; reserving 7521 MB for main workspace.
1560 variants loaded from .bim file.
3074 people (1721 mal

# Get and set rvtest command

In [14]:
#copy the excutable from https://github.com/zhanxw/rvtests/releases and untar it
rvtestCmd='/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest'

# Get the refFlat file for rvtests

In [15]:
cmd = 'gsutil cp gs://' + googleGSId+'/notebooks/refFlat_hg38.txt.zip ' + dataDir
shell_do(cmd)
#unzip
#cmd = ''+ dataDir+'refFlat_hg38.txt.zip'

Executing: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/refFlat_hg38.txt.zip /home/jupyter-user/notebooks/LSDProject/data/
Copying gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/refFlat_hg38.txt.zip...
/ [1 files][  7.2 MiB/  7.2 MiB]                                                
Operation completed over 1 objects/7.2 MiB.                                      


# Make rvtest phenotype file

In [14]:
#read the pheno file
inFile=dataDir + 'plink_pheno_withPCs.tab'
tmpDf=pd.read_csv(inFile, sep="\t")
v = tmpDf[['FID','IID','sex','PHENO','AgeBaseLine','race','PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9',
       'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18','PC19', 'PC20']]
#add 2 columns PATID and MATID
v.insert(2, "PATID", 'NA', True)
v.insert(3, "MATID", 'NA', True)
#print('\n v\n',v.head())
print('dim:', v.shape)
#filter race to white only if wanted
#v=v[v['race']=='White']
print('dim:', v.shape)

#write
fileName=dataDir + 'plink_pheno_withPCs_rvtestFormat_allRaces.tab'
print('fileName:', fileName)
with open(fileName, 'w') as f:
    f.write(v.to_csv(index=False, sep='\t'))
    
print('Done')



dim: (2835, 28)
dim: (2835, 28)
fileName: /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab
Done


# Run RVTests for one gene no maf threshold on all variants

In [43]:
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LSDProject/tools/rvtest'
phenoFile=dataDir + 'plink_pheno_withPCs_rvtestFormat.tab'
#phenoFile=dataDir+'pheno_head29.tab'

COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
#covFileName='/home/jupyter-user/notebooks/LaurieRobak1/data/test1.tab'
vcfFile=dataDir+ 'LSDGenes-WGS-VCF.vcf.gz'
geneName='GBA'
outPrefix=resDir+'LSDGenes-WGS-AMPPD_BURDEN_gene_'+geneName+'_NOFREQTHRESHOLD'
shell_do(f'{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} \
--pheno {phenoFile} \
--pheno-name PHENO \
--covar {phenoFile} \
--covar-name {COV_NAMES} \
--kernel skat,skato --geneFile {dataDir}refFlat_hg38.txt \
--gene {geneName} \
--out {outPrefix}')


print('Done!!')        

Executing: /home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --covar-name sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --geneFile /home/jupyter-user/notebooks/LSDProject/data/refFlat_hg38.txt --gene GBA --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes-WGS-AMPPD_BURDEN_gene_GBA_NOFREQTHRESHOLD.txt
Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please submit at: https://github.com/zhanxw/rvte

# Copy set file for LSD genes

In [54]:
cmd = 'gsutil cp gs://' + googleGSId+'/notebooks/LSD-54-genes_rvtest_SetFile.txt ' + dataDir
print('cmd:', cmd)
shell_do(cmd)


cmd = 'gsutil cp gs://' + googleGSId+'/notebooks/LSD-53-genes-withoutGBA_rvtest_SetFile.txt ' + dataDir
print('cmd:', cmd)
shell_do(cmd)


cmd: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-54-genes_rvtest_SetFile.txt /home/jupyter-user/notebooks/LSDProject/data/
Executing: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-54-genes_rvtest_SetFile.txt /home/jupyter-user/notebooks/LSDProject/data/
Copying gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-54-genes_rvtest_SetFile.txt...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 1 objects/1.4 KiB.                                      
cmd: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-53-genes-withoutGBA_rvtest_SetFile.txt /home/jupyter-user/notebooks/LSDProject/data/
Executing: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-53-genes-withoutGBA_rvtest_SetFile.txt /home/jupyter-user/notebooks/LSDProject/data/
Copying gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD-53-g

# Run RVTests for LSD genes no maf threshold on all variants (NO NEED TAKES time so go to P2 of code in R code)

In [48]:
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'

COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
vcfFile=dataDir+ 'LSDGenes-WGS-VCF.vcf.gz'
outPrefix=resDir+'LSDGenes_WGS_BURDEN_AllVariants.NOFREQTHRESHOLD'

setFile=dataDir+'LSD-54-genes_rvtest_SetFile.txt'
setName='LSDGenes54'

shell_do(f'{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} \
--pheno {phenoFile} \
--pheno-name PHENO \
--covar {phenoFile} \
--covar-name {COV_NAMES} \
--kernel skat,skato \
--setFile {setFile} \
--set {setName} \
--out {outPrefix}')


print('Done!!')  

Executing: /home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --covar-name sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --setFile /home/jupyter-user/notebooks/LSDProject/data/LSD-54-genes_rvtest_SetFile.txt --set LSDGenes54 --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes_WGS_BURDEN_AllVariants.NOFREQTHRESHOLD
Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please submit at: https://github.co

# MAF < 0.01 on All LSD genes variants

In [49]:
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'
COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
vcfFile=dataDir+ 'LSDGenes-WGS-VCF.vcf.gz'
outPrefix=resDir + 'LSDGenes_WGS_AMPPD_BURDEN_All_Variants.maf001'

setFile=dataDir+'/'+'LSD-54-genes_rvtest_SetFile.txt'
setName='LSDGenes54'

mafCut=0.01

shell_do(f'{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} \
--pheno {phenoFile} \
--pheno-name PHENO \
--covar {phenoFile} \
--covar-name {COV_NAMES} \
--kernel skat,skato \
--setFile {setFile} \
--set {setName} \
--freqUpper {mafCut} \
--out {outPrefix}')


print('Done!!')  

Executing: /home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-VCF.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --covar-name sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --setFile /home/jupyter-user/notebooks/LSDProject/data//LSD-54-genes_rvtest_SetFile.txt --set LSDGenes54 --freqUpper 0.01 --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes_WGS_AMPPD_BURDEN_All_Variants.maf001
Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please submit at: h

# MAF < 0.01 on CODING variants

In [51]:
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'

COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
vcfFile=dataDir+ 'LSDGenes-WGS-AMPPD_exonic.vcf.gz'
outPrefix=resDir + 'LSDGenes_WGS_AMPPD_BURDEN_Coding_Variants.maf001'

setFile=dataDir+'LSD-54-genes_rvtest_SetFile.txt'
setName='LSDGenes54'

mafCut=0.01

shell_do(f'{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} \
--pheno {phenoFile} \
--pheno-name PHENO \
--covar {phenoFile} \
--covar-name {COV_NAMES} \
--kernel skat,skato \
--setFile {setFile} \
--set {setName} \
--freqUpper {mafCut} \
--out {outPrefix}')





Executing: /home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_exonic.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --covar-name sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --setFile /home/jupyter-user/notebooks/LSDProject/data/LSD-54-genes_rvtest_SetFile.txt --set LSDGenes54 --freqUpper 0.01 --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes_WGS_AMPPD_BURDEN_Coding_Variants.maf001
Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please 

# MAF < 0.03 on CODING variants

In [53]:
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'

COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
vcfFile=dataDir+ 'LSDGenes-WGS-AMPPD_exonic.vcf.gz'
outPrefix=resDir + 'LSDGenes_WGS_AMPPD_BURDEN_Coding_Variants.maf003'

setFile=dataDir+'LSD-54-genes_rvtest_SetFile.txt'
setName='LSDGenes54'

mafCut=0.03

shell_do(f'{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} \
--pheno {phenoFile} \
--pheno-name PHENO \
--covar {phenoFile} \
--covar-name {COV_NAMES} \
--kernel skat,skato \
--setFile {setFile} \
--set {setName} \
--freqUpper {mafCut} \
--out {outPrefix}')

Executing: /home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_exonic.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --covar-name sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --setFile /home/jupyter-user/notebooks/LSDProject/data/LSD-54-genes_rvtest_SetFile.txt --set LSDGenes54 --freqUpper 0.03 --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes_WGS_AMPPD_BURDEN_Coding_Variants.maf003
Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please s

# MAF < 0.01 on CODING variants without GBA

In [56]:
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'

COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
vcfFile=dataDir+ 'LSDGenes-WGS-AMPPD_exonic.vcf.gz'
outPrefix=resDir + 'LSDGenes-withoutGBA_WGS_AMPPD_BURDEN_Coding_Variants.maf001'

setFile=dataDir+'LSD-53-genes-withoutGBA_rvtest_SetFile.txt'
setName='LSDGenes53'

mafCut=0.01

shell_do(f'{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} \
--pheno {phenoFile} \
--pheno-name PHENO \
--covar {phenoFile} \
--covar-name {COV_NAMES} \
--kernel skat,skato \
--setFile {setFile} \
--set {setName} \
--freqUpper {mafCut} \
--out {outPrefix}')

Executing: /home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_exonic.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --covar-name sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --setFile /home/jupyter-user/notebooks/LSDProject/data/LSD-53-genes-withoutGBA_rvtest_SetFile.txt --set LSDGenes53 --freqUpper 0.01 --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes-withoutGBA_WGS_AMPPD_BURDEN_Coding_Variants.maf001
Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feat

# MAF < 0.03 on CODING variants without GBA

In [57]:
#workingDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'

COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
#vcfFile=workingDir+'/'+ 'LSDGenes_WGS_AMPPD_exonic.vcf.gz'
vcfFile=dataDir+ 'LSDGenes-WGS-AMPPD_exonic.vcf.gz'
outPrefix=resDir+'LSDGenes-withoutGBA_WGS_AMPPD_BURDEN_Coding_Variants.maf003'

setFile=dataDir+'LSD-53-genes-withoutGBA_rvtest_SetFile.txt'
setName='LSDGenes53'

mafCut=0.03

shell_do(f'{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} \
--pheno {phenoFile} \
--pheno-name PHENO \
--covar {phenoFile} \
--covar-name {COV_NAMES} \
--kernel skat,skato \
--setFile {setFile} \
--set {setName} \
--freqUpper {mafCut} \
--out {outPrefix}')

Executing: /home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_exonic.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab --covar-name sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --setFile /home/jupyter-user/notebooks/LSDProject/data/LSD-53-genes-withoutGBA_rvtest_SetFile.txt --set LSDGenes53 --freqUpper 0.03 --out /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSDGenes-withoutGBA_WGS_AMPPD_BURDEN_Coding_Variants.maf003
Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feat

In [59]:
#shell_do('pwd')
#inFile='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes/LSDGenes_WGS_AMPPD_BURDEN.NOFREQTHRESHOLD.Skat.assoc'
#inFile='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes/rvtest/LSDGenes_WGS_AMPPD_BURDEN_Coding_Variants.maf003.SkatO.assoc'
inFile=resDir+'LSDGenes-withoutGBA_WGS_AMPPD_BURDEN_Coding_Variants.maf001.SkatO.assoc'
tmpDf=pd.read_csv(inFile, sep="\t")
print(tmpDf.shape)
print(tmpDf.head())
print('Done')

(1, 8)
        Range                                              RANGE  \
0  LSDGenes53  1:16985958-17011928,1:23845077-23868294,1:4007...   

   N_INFORMATIVE  NumVar  NumPolyVar       Q  rho    Pvalue  
0           2658    1299        1034  327339    0  0.218319  
Done
